In [1]:
import os 
import numpy as np
import pandas as pd
import torch
import math
from adapt.translator import usc_data_translator

%reload_ext autoreload
%autoreload 2

In [2]:
lt_logs_df = pd.read_csv('./logs/long_term_forecast.csv', index_col = 0)
from types import SimpleNamespace
ett_args = SimpleNamespace(**lt_logs_df.loc[2].to_dict())
usc_args = SimpleNamespace(**lt_logs_df.loc[26].to_dict())
bball_args = SimpleNamespace(**lt_logs_df.loc[106].to_dict())
ett_args.batch_size = 25
usc_args.batch_size = 25
bball_args.batch_size = 25

usc_args.input_features = 'P'

In [3]:
usc_args.d_model

256

In [4]:
bball = np.load('./dataset/bball/all_data.npy')
bball.shape

(117467, 50, 22)

In [5]:
bball[0]

array([[36.507, 39.35 , 18.69 , ..., 16.602, 21.337, 23.843],
       [34.941, 38.831, 16.745, ..., 16.601, 20.07 , 24.229],
       [33.812, 38.373, 15.033, ..., 16.47 , 19.097, 24.51 ],
       ...,
       [31.99 , 40.147, 35.023, ..., 33.046, 19.141, 19.01 ],
       [31.484, 40.863, 34.938, ..., 34.167, 18.908, 19.443],
       [30.415, 41.189, 34.818, ..., 35.066, 18.811, 19.877]])

### ETT

In [88]:
from data_provider.data_loader import Dataset_ETT_minute
from data_provider.data_factory import data_provider

ettds, ettdl = data_provider(args = ett_args, flag = 'train')

train 34129


In [89]:
ettds.data_x.shape

(34560, 7)

In [90]:
ettds.__len__(), ettdl.__len__(), ettdl.batch_size

(34129, 1366, 25)

In [91]:
ettds.__getitem__(0)[2].shape

(96, 4)

In [92]:
x,y, x_mark, y_mark = next(iter(ettdl))

In [93]:
y_mark[0].shape

torch.Size([384, 4])

### USC

In [94]:
uscds, uscdl = data_provider(args= usc_args, flag = 'train')
np.isnan(uscds.data_x).any(), np.isnan(uscds.data_y).any(), uscds.data_x.shape , uscds.data_x.min(), uscds.data_x.max()

train 23019


(False, False, (23019, 64, 32), -11.205336, 9.347016)

In [95]:
x,y, x_mark, y_mark = next(iter(uscdl))

In [96]:
import torch
x.shape
tmp = []
for i in range(x.shape[0]) :
    agent_data = [x[i, :, 2 + p :4 + p ] for p in range(15)] 
    #agent_data = torch.cat([x[i, :, 2 + p :4 + p ].reshape(1,x.shape[1],2) for p in range(15)]) 
    d = [ ad[-1,:] - ad[-2,:] for ad in agent_data]
    degree = [math.atan2(dp[1], dp[0]) for dp in d]
    prev_positions = torch.cat([torch.Tensor([ad[-1,0], ad[-1,1],ad[-2,0], ad[-2,1]]).reshape(1,4) for ad in agent_data])
    meta_info = torch.cat((torch.Tensor(degree).reshape(15,1), prev_positions),1)
    
    

In [97]:
meta_info.shape

torch.Size([15, 5])

In [98]:
len(agent_data), agent_data[0].shape

(15, torch.Size([64, 2]))

In [100]:
test = usc_data_translator(x, args = usc_args)

In [109]:
agent_data = [test[i]["agent_data"] for i in range(25)]
len(agent_data), len(agent_data[23]), agent_data[23][11].shape

(25, 15, torch.Size([64, 2]))

In [101]:
for k,v in test[0].items() :
    print(k, type(v))

agent_data <class 'list'>
lane_data <class 'list'>
city_name <class 'NoneType'>
file_name <class 'NoneType'>
origin_labels <class 'NoneType'>
labels <class 'torch.Tensor'>
label_is_valid <class 'torch.Tensor'>
consider <class 'torch.Tensor'>
cent_x <class 'NoneType'>
cent_y <class 'NoneType'>
angle <class 'NoneType'>
meta_info <class 'torch.Tensor'>


### USC Summary & model analysis

In [60]:
x_coords = [False]*3 + [i %4 == 3 for i in range(3, 63)]
y_coords = [False]*3 + [i %4 == 0 for i in range(3, 63)]
vx_coords = [False]*3 + [i %4 == 1 for i in range(3, 63)]
vy_coords = [False]*3 + [i %4 == 2 for i in range(3, 63)]
stats = []
for i, f in enumerate(os.listdir(usc_args.root_path)):
    print(f" Processing file {i+1} out of {len(os.listdir(usc_args.root_path))}", end = '\r')
    tmp = np.load(os.path.join(usc_args.root_path, f))
    x_min, x_max, y_min, y_max = tmp[:,:,x_coords].min(), tmp[:,:,x_coords].max(), tmp[:,:,y_coords].min(), tmp[:,:,y_coords].max()
    vx_min, vx_max, vy_min, vy_max = tmp[:,:,vx_coords].min(), tmp[:,:,vx_coords].max(), tmp[:,:,vy_coords].min(), tmp[:,:,vy_coords].max()
    g_min, g_max = tmp[:,:,1:].min(), tmp[:,:,1:].max()
    stats.append({'f':f, 'shape':tmp.shape, 'min':g_min, 'max':g_max, 
                  'x_min':x_min, 'x_max':x_max, 'y_min':y_min, 'y_max':y_max, 
                  'vx_min':vx_min, 'vx_max':vx_max, 'vy_min':vy_min, 'vy_max':vy_max})
pd.DataFrame(stats)

,f,shape,min,max,x_min,x_max,y_min,y_max,vx_min,vx_max,vy_min,vy_max
0,PVTO_seq48_tar48_X.npy,"(46530, 48, 63)",-32.532787,129.939774,-29.809175,129.939774,-19.990734,89.974266,-2.993788,2.971190,-2.94513,2.968086
1,PVTO_seq32_tar32_y.npy,"(56173, 32, 63)",-57.081276,129.945679,-29.416340,129.945679,-19.967653,89.974266,-2.984167,2.971190,-2.94513,2.994125
2,PVTO_seq24_tar24_X.npy,"(61648, 24, 63)",-29.809175,129.939774,-29.809175,129.939774,-19.990734,89.974266,-2.993788,2.971190,-2.94513,2.994125
3,PVTO_seq24_tar24_y.npy,"(61648, 24, 63)",-57.081276,129.945679,-29.416340,129.945679,-19.988234,89.974266,-2.984167,2.971190,-2.94513,2.994125
4,PVTO_seq32_tar32_X.npy,"(56173, 32, 63)",-29.809175,129.939774,-29.809175,129.939774,-19.990734,89.974266,-2.993788,2.971190,-2.94513,2.968086
5,PVTO_seq16_tar16_X.npy,"(67701, 16, 63)",-29.809175,129.952011,-29.809175,129.952011,-19.990734,89.974266,-2.993788,2.971190,-2.94513,2.994125
6,PVTO_seq64_tar64_y.npy,"(38366, 64, 63)",-57.081276,129.880692,-29.416340,129.880692,-18.236652,89.974266,-2.984167,2.947705,-2.94513,2.968086
7,PVTO_seq48_tar48_y.npy,"(46530, 48, 63)",-57.081276,129.945679,-29.416340,129.945679,-19.967653,89.974266,-2.984167,2.971190,-2.94513,2.994125
8,PVTO_seq16_tar16_y.npy,"(67701, 16, 63)",-57.081276,129.945679,-29.416340,129.945679,-19.988234,89.974266,-2.984167,2.971190,-2.94513,2.994125
9,PVTO_seq64_tar64_X.npy,"(38366, 64, 63)",-35.836102,129.939774,-29.809175,129.939774,-18.236652,89.974266,-2.993788,2.971190,-2.94513,2.968086


In [71]:
from models.LSTransformer import Model
from utils.model_wrapper import ModelWrapper
usc_args.seq_len, usc_args.pred_len = 24,24
model = ModelWrapper(Model(configs=usc_args), usc_args)
model.summary()

Model quick summary
Layer name                    |          Nb units|      Nb tr params|  Nb non tr params
LSTM_0                        |               256|           296 960|                 0
Linear_0                      |               256|             8 448|                 0
LayerNorm_0                   |                 -|               512|                 0
Linear_1                      |               256|            65 792|                 0
TransformerEncoderLayer_0     |               256|         1 315 072|                 0
Linear_2                      |               256|            65 792|                 0
LayerNorm_1                   |                 -|               512|                 0
Linear_3                      |               256|            65 792|                 0
LayerNorm_2                   |                 -|               512|                 0
Linear_4                      |                24|             6 168|                 0
Skip_connect